In [1]:
import sys
import time
from os import listdir
from os.path import isfile, join
import string
import numpy as np
from sklearn.linear_model import LogisticRegression as LR
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import EnglishStemmer
from collections import OrderedDict
import pandas as pd
from nltk.corpus import stopwords

In [2]:
# imports needed and logging
import gzip
import gensim 
import logging
from gensim.models import Word2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
def get_file_names(path):
    return [path + "/" + f for f in listdir(path) if isfile(join(path, f))]

In [4]:
def file_to_tokens(file_name):#, min_word_size, stemmer=None):
    with open(file_name) as f:
        file_string = f.read().rstrip()

        return gensim.utils.simple_preprocess(file_string)
        #return file_string

In [5]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])

## Train Word Embeddings


In [6]:
def build_data(pos_file_path, neg_file_path ):
    pos_file_names = get_file_names(pos_file_path)
    neg_file_names = get_file_names(neg_file_path)
    # m is the number of training examples
    m_pos = len(pos_file_names)
    m_neg = len(neg_file_names)
    m = m_pos + m_neg
    print(m)
    pos_labels = np.ones(m_pos)
    neg_labels = np.zeros(m_neg)
    y = np.concatenate((pos_labels, neg_labels), axis=0)
    pos_data=pd.DataFrame(index=list(range(m_pos)),columns=['Review'])
    neg_data=pd.DataFrame(index=list(range(m_neg)),columns=['Review'])
    for i in range(m_pos):
        #pos_data.loc[i,'Review']=text_process(file_to_tokens(pos_file_names[i]))
        pos_data.loc[i,'Review']=file_to_tokens(pos_file_names[i])
    pos_data['Label'] = pos_labels
    for i in range(m_neg):
        #neg_data.loc[i,'Review']=text_process(file_to_tokens(neg_file_names[i]))
        neg_data.loc[i,'Review']=file_to_tokens(neg_file_names[i])
    neg_data['Label'] = neg_labels
    data=pd.concat([pos_data,neg_data],axis=0)
    data=data.sample(frac=1).reset_index(drop=True)
    return data



In [7]:
data_small = build_data('review_polarity_data/pos', 'review_polarity_data/neg' )
data_big_part1 = build_data ('aclImdb/train/pos','aclImdb/train/neg')
data_big_part2 = build_data ('aclImdb/test/pos','aclImdb/test/neg')
data_big = pd.concat([data_big_part1,data_big_part2],axis=0)
data_big = data_big.sample(frac=1).reset_index(drop=True)
data_huge = pd.concat([data_small,data_big],axis=0)
data_huge = data_huge.sample(frac=1).reset_index(drop=True)


2000
25000
25000


In [8]:
len(data_huge)

52000

In [10]:
# build vocabulary and train model
model = gensim.models.Word2Vec(
    data_huge['Review'],
    size=100,
    window=10,
    min_count=2,
    workers=5,
    sg = 1)
model.train(data_huge['Review'], total_examples=len(data_huge['Review']), epochs=100)

2019-02-09 10:18:22,355 : INFO : collecting all words and their counts
2019-02-09 10:18:22,360 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-02-09 10:18:22,800 : INFO : PROGRESS: at sentence #10000, processed 2389649 words, keeping 53141 word types
2019-02-09 10:18:23,202 : INFO : PROGRESS: at sentence #20000, processed 4774732 words, keeping 70918 word types
2019-02-09 10:18:23,601 : INFO : PROGRESS: at sentence #30000, processed 7139703 words, keeping 83445 word types
2019-02-09 10:18:24,005 : INFO : PROGRESS: at sentence #40000, processed 9546402 words, keeping 93848 word types
2019-02-09 10:18:24,399 : INFO : PROGRESS: at sentence #50000, processed 11942325 words, keeping 102838 word types
2019-02-09 10:18:24,486 : INFO : collected 104557 word types from a corpus of 12431273 raw words and 52000 sentences
2019-02-09 10:18:24,487 : INFO : Loading a fresh vocabulary
2019-02-09 10:18:25,501 : INFO : effective_min_count=2 retains 65356 unique words (62%

2019-02-09 10:19:22,944 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-09 10:19:22,944 : INFO : EPOCH - 1 : training on 12431273 raw words (9483073 effective words) took 56.3s, 168323 effective words/s
2019-02-09 10:19:23,999 : INFO : EPOCH 2 - PROGRESS: at 1.50% examples, 135784 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:19:25,026 : INFO : EPOCH 2 - PROGRESS: at 2.86% examples, 129701 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:19:26,133 : INFO : EPOCH 2 - PROGRESS: at 4.27% examples, 124701 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:19:27,160 : INFO : EPOCH 2 - PROGRESS: at 5.94% examples, 131494 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:19:28,228 : INFO : EPOCH 2 - PROGRESS: at 7.67% examples, 136181 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:19:29,232 : INFO : EPOCH 2 - PROGRESS: at 9.51% examples, 141894 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:19:30,248 : INFO : EPOCH 2 - PROGRESS: at 11.17% examples, 143700 words/s, in_qsize 

2019-02-09 10:20:33,561 : INFO : EPOCH 3 - PROGRESS: at 19.24% examples, 159544 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:20:34,569 : INFO : EPOCH 3 - PROGRESS: at 21.05% examples, 160373 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:20:35,613 : INFO : EPOCH 3 - PROGRESS: at 22.73% examples, 159618 words/s, in_qsize 10, out_qsize 0
2019-02-09 10:20:36,668 : INFO : EPOCH 3 - PROGRESS: at 24.56% examples, 159819 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:20:37,678 : INFO : EPOCH 3 - PROGRESS: at 26.37% examples, 160941 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:20:38,748 : INFO : EPOCH 3 - PROGRESS: at 28.14% examples, 160481 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:20:39,871 : INFO : EPOCH 3 - PROGRESS: at 30.04% examples, 160343 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:20:40,873 : INFO : EPOCH 3 - PROGRESS: at 31.81% examples, 160986 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:20:41,973 : INFO : EPOCH 3 - PROGRESS: at 33.73% examples, 161136 words/s, in_qsi

2019-02-09 10:21:45,471 : INFO : EPOCH 4 - PROGRESS: at 43.52% examples, 163159 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:21:46,498 : INFO : EPOCH 4 - PROGRESS: at 45.27% examples, 163016 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:21:47,561 : INFO : EPOCH 4 - PROGRESS: at 47.00% examples, 162694 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:21:48,596 : INFO : EPOCH 4 - PROGRESS: at 48.72% examples, 162537 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:21:49,605 : INFO : EPOCH 4 - PROGRESS: at 50.45% examples, 162597 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:21:50,657 : INFO : EPOCH 4 - PROGRESS: at 52.23% examples, 162382 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:21:51,695 : INFO : EPOCH 4 - PROGRESS: at 54.05% examples, 162496 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:21:52,703 : INFO : EPOCH 4 - PROGRESS: at 55.83% examples, 162523 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:21:53,708 : INFO : EPOCH 4 - PROGRESS: at 57.55% examples, 162378 words/s, in_qsiz

2019-02-09 10:22:55,989 : INFO : EPOCH 5 - PROGRESS: at 60.49% examples, 153246 words/s, in_qsize 8, out_qsize 1
2019-02-09 10:22:57,101 : INFO : EPOCH 5 - PROGRESS: at 62.09% examples, 152718 words/s, in_qsize 10, out_qsize 0
2019-02-09 10:22:58,151 : INFO : EPOCH 5 - PROGRESS: at 63.83% examples, 152814 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:22:59,226 : INFO : EPOCH 5 - PROGRESS: at 65.42% examples, 152594 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:23:00,296 : INFO : EPOCH 5 - PROGRESS: at 67.24% examples, 152811 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:23:01,374 : INFO : EPOCH 5 - PROGRESS: at 69.01% examples, 152968 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:23:02,394 : INFO : EPOCH 5 - PROGRESS: at 70.74% examples, 153315 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:23:03,401 : INFO : EPOCH 5 - PROGRESS: at 72.45% examples, 153526 words/s, in_qsize 9, out_qsize 0
2019-02-09 10:23:04,424 : INFO : EPOCH 5 - PROGRESS: at 74.21% examples, 153666 words/s, in_qsi

...

2019-02-09 12:09:25,122 : INFO : EPOCH 97 - PROGRESS: at 81.80% examples, 122392 words/s, in_qsize 8, out_qsize 1
2019-02-09 12:09:26,149 : INFO : EPOCH 97 - PROGRESS: at 83.75% examples, 123248 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:09:27,438 : INFO : EPOCH 97 - PROGRESS: at 85.17% examples, 122888 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:09:28,950 : INFO : EPOCH 97 - PROGRESS: at 87.12% examples, 122900 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:09:30,340 : INFO : EPOCH 97 - PROGRESS: at 89.01% examples, 123015 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:09:31,429 : INFO : EPOCH 97 - PROGRESS: at 90.52% examples, 123137 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:09:32,461 : INFO : EPOCH 97 - PROGRESS: at 91.77% examples, 123035 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:09:33,465 : INFO : EPOCH 97 - PROGRESS: at 93.01% examples, 122992 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:09:34,766 : INFO : EPOCH 97 - PROGRESS: at 94.70% examples, 123065 words/s

2019-02-09 12:10:40,268 : INFO : EPOCH 98 - PROGRESS: at 80.32% examples, 123084 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:10:41,508 : INFO : EPOCH 98 - PROGRESS: at 81.82% examples, 122915 words/s, in_qsize 10, out_qsize 0
2019-02-09 12:10:42,509 : INFO : EPOCH 98 - PROGRESS: at 83.36% examples, 123225 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:10:43,521 : INFO : EPOCH 98 - PROGRESS: at 84.54% examples, 123042 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:10:44,528 : INFO : EPOCH 98 - PROGRESS: at 85.74% examples, 122871 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:10:45,732 : INFO : EPOCH 98 - PROGRESS: at 86.97% examples, 122446 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:10:47,079 : INFO : EPOCH 98 - PROGRESS: at 88.84% examples, 122648 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:10:48,124 : INFO : EPOCH 98 - PROGRESS: at 90.27% examples, 122744 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:10:49,127 : INFO : EPOCH 98 - PROGRESS: at 91.53% examples, 122699 words/

2019-02-09 12:11:53,437 : INFO : EPOCH 99 - PROGRESS: at 77.73% examples, 126892 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:11:54,451 : INFO : EPOCH 99 - PROGRESS: at 78.65% examples, 126113 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:11:55,636 : INFO : EPOCH 99 - PROGRESS: at 80.48% examples, 126472 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:11:56,827 : INFO : EPOCH 99 - PROGRESS: at 81.97% examples, 126338 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:11:57,935 : INFO : EPOCH 99 - PROGRESS: at 83.83% examples, 126863 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:11:58,953 : INFO : EPOCH 99 - PROGRESS: at 85.17% examples, 126838 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:12:00,022 : INFO : EPOCH 99 - PROGRESS: at 86.53% examples, 126701 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:12:01,048 : INFO : EPOCH 99 - PROGRESS: at 87.89% examples, 126772 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:12:02,233 : INFO : EPOCH 99 - PROGRESS: at 89.25% examples, 126414 words/s

2019-02-09 12:13:08,007 : INFO : EPOCH 100 - PROGRESS: at 76.40% examples, 125232 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:13:09,037 : INFO : EPOCH 100 - PROGRESS: at 77.56% examples, 124949 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:13:10,083 : INFO : EPOCH 100 - PROGRESS: at 78.92% examples, 124778 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:13:11,106 : INFO : EPOCH 100 - PROGRESS: at 79.69% examples, 123898 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:13:12,422 : INFO : EPOCH 100 - PROGRESS: at 81.46% examples, 123907 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:13:13,510 : INFO : EPOCH 100 - PROGRESS: at 82.87% examples, 123919 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:13:14,581 : INFO : EPOCH 100 - PROGRESS: at 84.62% examples, 124426 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:13:15,612 : INFO : EPOCH 100 - PROGRESS: at 86.05% examples, 124530 words/s, in_qsize 9, out_qsize 0
2019-02-09 12:13:16,622 : INFO : EPOCH 100 - PROGRESS: at 87.34% examples, 12454

(948283611, 1243127300)

In [11]:
model.save("word2vec4_huge_data_100,10,2,5,100_skip.model")

2019-02-09 12:13:26,434 : INFO : saving Word2Vec object under word2vec4_huge_data_100,10,2,5,100_skip.model, separately None
2019-02-09 12:13:26,436 : INFO : not storing attribute vectors_norm
2019-02-09 12:13:26,470 : INFO : not storing attribute cum_table
2019-02-09 12:13:27,576 : INFO : saved word2vec4_huge_data_100,10,2,5,100_skip.model


In [12]:
model = Word2Vec.load("word2vec4_huge_data_100,10,2,5,100_skip.model")

2019-02-09 12:13:27,583 : INFO : loading Word2Vec object from word2vec4_huge_data_100,10,2,5,100_skip.model
2019-02-09 12:13:28,818 : INFO : loading wv recursively from word2vec4_huge_data_100,10,2,5,100_skip.model.wv.* with mmap=None
2019-02-09 12:13:28,819 : INFO : setting ignored attribute vectors_norm to None
2019-02-09 12:13:28,820 : INFO : loading vocabulary recursively from word2vec4_huge_data_100,10,2,5,100_skip.model.vocabulary.* with mmap=None
2019-02-09 12:13:28,821 : INFO : loading trainables recursively from word2vec4_huge_data_100,10,2,5,100_skip.model.trainables.* with mmap=None
2019-02-09 12:13:28,822 : INFO : setting ignored attribute cum_table to None
2019-02-09 12:13:28,824 : INFO : loaded word2vec4_huge_data_100,10,2,5,100_skip.model


## NLP_Task : now let's use our pretrained word embedding to accomplish our task

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression,RidgeClassifier,Lasso
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer,f1_score, auc, roc_auc_score
from sklearn.model_selection import ShuffleSplit

/home/ahmedbahgat/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ahmedbahgat/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [14]:
word_vectors = model.wv

In [15]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/home/ahmedbahgat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [16]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.values())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [17]:
data_huge_train, data_huge_test, label_huge_train, label_huge_test = \
train_test_split(data_huge['Review'], data_huge['Label'], test_size=0.2, random_state = 0)
print(len(data_huge_train), len(data_huge_test), len(data_huge_train) + len(data_huge_test))

41600 10400 52000


In [18]:

pipeline1 = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),  # strings to token integer counts
    ('classifier', RidgeClassifier(random_state=0))
    #('classifier', xgb.XGBClassifier(random_state=0))
])



In [20]:
pipeline1.fit(data_huge_train,label_huge_train)

Pipeline(memory=None,
     steps=[('word2vec vectorizer', <__main__.MeanEmbeddingVectorizer object at 0x7fef20cd57b8>), ('classifier', RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=0, solver='auto',
        tol=0.001))])

In [21]:
predictions1 = pipeline1.predict(data_huge_test)

In [22]:
print(roc_auc_score(predictions1,label_huge_test))

0.8802839929819632


In [23]:
print(classification_report(predictions1,label_huge_test))

             precision    recall  f1-score   support

        0.0       0.88      0.88      0.88      5223
        1.0       0.88      0.88      0.88      5177

avg / total       0.88      0.88      0.88     10400

